In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import time

camera = cv2.VideoCapture(2)
model = YOLO("yolov8n.pt")

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)
percentage_bar_annotator = sv.BoxAnnotator()

while True:
    success, frame = camera.read() 
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    coord_center = []
    for i in range (len(detections)):
        centers = (detections.xyxy[:,:2][i]+detections.xyxy[:,2:][i])/2
        coord_center.append(centers)
    coord_center=np.array(coord_center)  
    if coord_center.size != 0:
        print(coord_center)
        cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    frame = label_annotator.annotate(
        scene=frame,
        detections=detections
    )
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
Using TensorFlow backend.


[[     522.69      104.16]
 [     522.38       103.7]]
[[     522.73      104.37]
 [     522.44      103.88]]
[[     522.46      104.45]]
[[     522.25      103.94]]
[[      522.4      103.72]
 [     522.67      104.17]]
[[     522.32      103.83]]
[[     521.92      103.86]]
[[     522.83      104.31]]
[[     522.68         104]]
[[     522.64      103.99]]
[[     522.74      103.96]]
[[     522.68      104.15]
 [     522.37      103.72]]
[[     522.11      103.81]
 [     522.44      104.33]]
[[     522.55      104.02]
 [     522.88      104.55]]
[[     522.43      103.76]]
[[      522.2      103.72]]
[[     522.03      103.82]]
[[     522.55      103.89]]
[[     522.24      103.88]]
[[     523.19      105.14]]
[[     522.51      104.21]]
[[     522.35      103.67]
 [     522.68      104.08]]
[[      522.7      104.16]]
[[     522.72      103.97]]
[[     522.73      104.22]]
[[      522.3      103.88]
 [     522.65      104.38]]
[[     522.37      104.03]
 [     522.73      104.67]]
[

KeyboardInterrupt: 

In [1]:
import cv2
from realsense_camera import *
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import pyrealsense2
from kuka import Kuka
import time
from openshowvar import *


#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)

kuka.read_cartesian()
x0=282.73
y0=122.48
z0=451.23
# Построение траектории
trajectory_arr = []
# Построение квадрата
trajectory_arr.append(np.array([kuka.x_cartesian, kuka.y_cartesian, kuka.z_cartesian, kuka.A_cartesian, kuka.B_cartesian, kuka.C_cartesian]))
trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous_massiv(trajectory_arr)

fx=604.602
fy=604.162
Cx=320
Cy=240
cap = RealsenseCamera()
#camera = cv2.VideoCapture(0)
model = YOLO("yolov8n.pt")

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)
percentage_bar_annotator = sv.BoxAnnotator()
print('start')
while True:
    success, depth_colormap, frame, depth = cap.get_frame_stream()
    #success, frame = camera.read() 
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #classes = result[0].boxes.cls.tolist() #Класс
    #print(classes)
    coord_center = []
    for i in range (len(detections)):
        centers = (detections.xyxy[:,:2][i]+detections.xyxy[:,2:][i])/2
        coord_center.append(centers)
    coord_center=np.array(coord_center)  
    if coord_center.size != 0:
        print(coord_center)
        cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
        for i in range(len(detections)):
            X_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cx-coord_center[0][0])/fx)
            Y_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cy-coord_center[0][1])/fy)
            X_mm=X_mm-30
            Y_mm=Y_mm-65
            Z_mm=depth[int(coord_center[0][1]), int(coord_center[0][0])]-40-50
            print('Xmm',X_mm)
            print('Ymm',Y_mm)
            print('Zmm',Z_mm)
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    frame = label_annotator.annotate(
        scene=frame,
        detections=detections
    )
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

Using TensorFlow backend.


[[      404.3      242.89]]
Xmm -99.57649799992392
Ymm -67.38874822102343
Zmm 409
[[     404.11      243.08]]
Xmm -99.41945539552452
Ymm -67.54559012652481
Zmm 409
[[     404.33      243.03]]
Xmm -99.60067777542086
Ymm -67.5044167633048
Zmm 409
[[     404.01      242.91]]
Xmm -99.33815090041608
Ymm -67.40529570370109
Zmm 409
[[     404.04      242.74]]
Xmm -99.36235586317916
Ymm -67.25965769165015
Zmm 409
[[     404.15      242.87]]
Xmm -99.45489387898724
Ymm -67.37061278494336
Zmm 409
[[     404.08      243.06]]
Xmm -99.39016260500064
Ymm -67.5241401527873
Zmm 409
[[     404.06      242.51]]
Xmm -99.37590661236389
Ymm -67.07211535370189
Zmm 409
[[     404.06      243.12]]
Xmm -99.37993657494673
Ymm -67.57530754019876
Zmm 409
[[      403.9      242.71]]
Xmm -99.2435475287843
Ymm -67.24186253129916
Zmm 409
[[     404.04       243.1]]
Xmm -99.35968601296804
Ymm -67.56073869787176
Zmm 409
[[     404.31      243.09]]
Xmm -99.58098133329732
Ymm -67.5556219591306
Zmm 409
[[     404.07      2

[[     404.05      243.18]]
Xmm -99.36603320403597
Ymm -67.62385354421573
Zmm 409
[[     404.19      242.87]]
Xmm -99.34290372239455
Ymm -67.36700661484743
Zmm 408
[[     404.31      243.02]]
Xmm -99.4460651112607
Ymm -67.48619146594335
Zmm 408
[[     403.91      242.79]]
Xmm -99.25049921423968
Ymm -67.30202832132436
Zmm 409
[[     404.09      243.02]]
Xmm -99.39860033915843
Ymm -67.49821618335248
Zmm 409
[[     404.15      243.16]]
Xmm -99.45423901006752
Ymm -67.60973840286086
Zmm 409
[[     403.88      242.99]]
Xmm -99.2313820792374
Ymm -67.47013713430009
Zmm 409
[[     404.37       243.3]]
Xmm -99.63332047234174
Ymm -67.72287378138121
Zmm 409
[[     404.27      243.22]]
Xmm -99.5516885427734
Ymm -67.66027565003326
Zmm 409
[[      404.3       243.1]]
Xmm -99.57856335574762
Ymm -67.56121760445343
Zmm 409
[[     404.18      243.41]]
Xmm -99.47600080801477
Ymm -67.8200162006343
Zmm 409
[[     404.23      243.22]]
Xmm -99.65636878940609
Ymm -67.66344746534637
Zmm 410
[[     404.08      2

KeyboardInterrupt: 

In [1]:
import cv2
from realsense_camera import *
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import pyrealsense2
from kuka import Kuka
import time
from openshowvar import *


#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(50)

kuka.read_cartesian()
x0=282.73
y0=122.48
z0=451.23
# Чтение текущих координат робота
kuka.read_cartesian()
# Построение траектории
first_point=[x0, y0, z0, 90.57, 0, 180]
trajectory=np.array([first_point])
kuka.lin_continuous(kuka,trajectory)

fx=604.602
fy=604.162
Cx=320
Cy=240
cap = RealsenseCamera()
#camera = cv2.VideoCapture(0)
model = YOLO("yolov8n.pt")

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)
percentage_bar_annotator = sv.BoxAnnotator()
print('start')
while True:
    success, depth_colormap, frame, depth = cap.get_frame_stream()
    #success, frame = camera.read() 
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #classes = result[0].boxes.cls.tolist() #Класс
    #print(classes)
    coord_center = []
    for i in range (len(detections)):
        centers = (detections.xyxy[:,:2][i]+detections.xyxy[:,2:][i])/2
        coord_center.append(centers)
    coord_center=np.array(coord_center)  
    if coord_center.size != 0:
        print(coord_center)
        cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
        for i in range(len(detections)):
            X_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cx-coord_center[0][0])/fx)
            Y_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cy-coord_center[0][1])/fy)
            X_mm=X_mm+30
            Y_mm=Y_mm+65
            Z_mm=depth[int(coord_center[0][1]), int(coord_center[0][0])]-40-50
            print('Xmm',X_mm)
            print('Ymm',Y_mm)
            print('Zmm',Z_mm)
            
            # Движение Kuka
            kuka.open_grip()
            # Чтение текущих координат робота
            kuka.read_cartesian()
            # Построение траектории
            first_point=[kuka.x_cartesian-X_mm, kuka.y_cartesian+Y_mm, kuka.z_cartesian-Z_mm,kuka.A_cartesian, 0, kuka.C_cartesian]
            trajectory=np.array([first_point])
            kuka.lin_continuous(kuka,trajectory)
            kuka.close_grip()
            time.sleep(1)
            # Чтение текущих координат робота
            kuka.read_cartesian()
            # Построение траектории
            first_point=[x0, y0, z0, 90.57, 0, 180]
            trajectory=np.array([first_point])
            kuka.lin_continuous(kuka,trajectory)
            
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    frame = label_annotator.annotate(
        scene=frame,
        detections=detections
    )
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

start


Using TensorFlow backend.


[[     404.14      243.19]]
Xmm -39.43998301743078
Ymm 62.36403516038958
Zmm 409
[[     214.43      259.57]]
Xmm 118.1790521325861
Ymm 48.646204752554596
Zmm 415


KeyboardInterrupt: 

In [4]:
import cv2
from realsense_camera import *
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import pyrealsense2
from kuka import Kuka
import time
from openshowvar import *


#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(50)

kuka.read_cartesian()
x0=282.73
y0=122.48
z0=451.23
# Чтение текущих координат робота
kuka.read_cartesian()
# Построение траектории
first_point=[x0, y0, z0, 90.57, 0, 180]
trajectory=np.array([first_point])
kuka.lin_continuous(kuka,trajectory)

fx=604.602
fy=604.162
Cx=320
Cy=240
cap = RealsenseCamera()
#camera = cv2.VideoCapture(0)
model = YOLO("yolov8n-seg.pt")

def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height
print('start')
while True:
    success, depth_colormap, frame, depth = cap.get_frame_stream()
    #success, frame = camera.read() 
    results = model(frame, verbose=False)
    boxes = results[0].boxes.xyxy.tolist() #Рамка
    #print(boxes)
    if boxes!=[]:
        frame=results[0].plot()
        classes = results[0].boxes.cls.tolist() #Класс
        #print(classes)
        confidences = results[0].boxes.conf.tolist()
        #print(confidences)
        mask_imshow = results[0].masks[0].data.cpu().numpy().transpose(1, 2, 0)#Маска двухбитная 
        mask1 = mask_imshow.astype(np.uint8)
        contours, _ = cv2.findContours(mask1, cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_NONE) 
        if len(contours) != 0: 
            maxc = max(contours, key=cv2.contourArea)
            cv2.drawContours(frame, maxc, -1, (0, 0, 255), 2)
        rotation,width,height=getOrientation(maxc, frame)
        print(rotation)
        coord_center = []
        for i in range (len(boxes)):
            x = (boxes[i][0]+boxes[i][2])/2
            y= (boxes[i][1]+boxes[i][3])/2
            coord_center.append([x,y])
        coord_center=np.array(coord_center)  
        if coord_center.size != 0:
            print(coord_center)
            cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
            for i in range(len(boxes)):
                X_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cx-coord_center[0][0])/fx)
                Y_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cy-coord_center[0][1])/fy)
                X_mm=X_mm+30
                Y_mm=Y_mm+65
                Z_mm=depth[int(coord_center[0][1]), int(coord_center[0][0])]-40-50
            
                print('Xmm',X_mm)
                print('Ymm',Y_mm)
                print('Zmm',Z_mm)
                
                # Движение Kuka
                kuka.open_grip()
                # Чтение текущих координат робота
                kuka.read_cartesian()
                # Построение траектории
                first_point=[kuka.x_cartesian-X_mm, kuka.y_cartesian+Y_mm, kuka.z_cartesian-Z_mm,kuka.A_cartesian, 0, kuka.C_cartesian]
                second_point=[kuka.x_cartesian-X_mm, kuka.y_cartesian+Y_mm, kuka.z_cartesian-Z_mm,rotation, 0, kuka.C_cartesian]
                trajectory=np.array([first_point, second_point])
                kuka.lin_continuous(kuka,trajectory)
                kuka.close_grip()
                time.sleep(1)
                # Чтение текущих координат робота
                kuka.read_cartesian()
                # Построение траектории
                first_point=[x0, y0, z0, 90.57, 0, 180]
                trajectory=np.array([first_point])
                kuka.lin_continuous(kuka,trajectory)
            
    cv2.imshow("mask_imshow", mask_imshow)
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

start
105
[[     213.46      260.38]]
Xmm 118.98659184890109
Ymm 47.96183229570463
Zmm 415
105
[[     213.64      261.75]]
Xmm 119.19244114136275
Ymm 46.74655292286245
Zmm 417


KeyboardInterrupt: 